https://python.langchain.com/docs/how_to/multimodal_inputs/#documents-from-a-url

In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Markdown

In [2]:
load_dotenv()

True

In [3]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai", )
# response = model.invoke("Hello World!!")
# print(response)

Gemini models are intelligent enough to take multimodal input with longer context window. This solves lot of problems and somehow elimintaes the need for RAG (not completely if its query intensive.) Here we just summarize the content once so RAG is not needed as such.

In [7]:
# Method 3: Using ChatPromptTemplate for multimodal input
def create_multimodal_template():
    """Create a ChatPromptTemplate that can handle multimodal input"""

    template = ChatPromptTemplate.from_messages([
        ("system", "You are an educational AI assistant. Analyze images/pdf and provide detailed explanations suitable for students."),
        ("human", [
            {"type": "text", "text": "{text_input}"},
            # {"type": "image_url", "image_url": {"url": "{image_url}"}}
            {"type": "file", "url": "{file_url}", "source_type": "url"}

        ])
    ])

    return template

text_input = "Describe this image"
image_url = ''

# Usage with template
multimodal_template = create_multimodal_template()

pdf_url = 'https://kclenjtsvdayeghmmmkm.supabase.co/storage/v1/object/public/learning-sources/public/Slides_Module8.pdf'
image_url = 'https://kclenjtsvdayeghmmmkm.supabase.co/storage/v1/object/public/learning-sources/public/maxresdefault.jpg'
audio_url = 'https://kclenjtsvdayeghmmmkm.supabase.co/storage/v1/object/public/learning-sources/public/speech_20250704112803322.mp3'

# Format the prompt
formatted_prompt = multimodal_template.format_messages(
    text_input="Summarise the content in the provided source",
    # image_url=""  # or base64 URL
    # file_url=image_url  # or base64 URL
    file_url=audio_url
)

response = model.invoke(formatted_prompt)
print(response.content)

ज़रूर, मैं आपकी मदद कर सकता हूँ। यहाँ स्रोत का सारांश दिया गया है:

यह पॉडकास्ट चोल साम्राज्य के बारे में बात करता है, जो लगभग 9वीं शताब्दी से 13वीं शताब्दी तक सत्ता में रहा और जिसने दक्षिण भारत को एक शक्तिशाली और समृद्ध साम्राज्य बनाया। उनका साम्राज्य तमिलनाडु से लेकर श्रीलंका और मालदीव तक फैला हुआ था।

चोल राजाओं ने एक मजबूत नौसेना बनाई जिसके कारण वे समुद्री व्यापार में भी आगे रहे। चोल साम्राज्य की सबसे खास बात थी उनका स्थानीय स्वशासन। उन्होंने गांवों में उर और सभा जैसी संस्थाएं बनाईं जो गांव के मामलों को खुद देखती थीं। इसके अलावा, चोल राजाओं ने कला और साहित्य को भी बहुत बढ़ावा दिया। उन्होंने बड़े-बड़े मंदिर बनवाए, जैसे कि तंजावुर का बृहदेश्वर मंदिर, जो आज भी उनकी महानता की कहानी कहता है।

चोल साम्राज्य के राजाओं में राजाराज चोल प्रथम और राजेंद्र चोल प्रथम सबसे प्रसिद्ध थे। राजाराज चोल प्रथम ने नौसेना की शक्ति का इस्तेमाल करके कई देशों को जीता, जबकि राजेंद्र चोल प्रथम ने गंगा नदी तक अपनी सेना भेजी और गंगैकोंड चोलपुरम नामक नई राजधानी बनाई।

लेकिन 13वीं शताब्दी में पांड्य राजाओं के आक्

## Building a multi-modal assistance for our use case

In [14]:
# Student profile type definition
class StudentProfile(TypedDict):
    gender: str
    class_level: str  # e.g., "class 6", "12th", "undergrad", "postgrad"
    language: str     # e.g., "hindi", "english", "marathi"


class UserPrompt(TypedDict):
    topic: str
    file_url: str
    # image_file: str
    # audio_file: str

# Graph state


class State(TypedDict):
    student_profile: StudentProfile
    user_prompt: UserPrompt
    summary_notes: str
    podcast_script: str
    mindmap: str
    quiz: str
    recommended_resources: str
    study_plan: str
    combined_output: str

In [ ]:
# Prompts Templates
class PersonalizedPromptsStudent:
    def __init__(self, topic, file_url: str, student_profile: dict, user_instructions: str = ""):
        self.topic = topic
        self.file_url = file_url
        self.student_profile = student_profile
        self.user_instructions = user_instructions

    def summary_notes(self):
        """Create a personalized prompt based on student profile"""

        template = ChatPromptTemplate([
            ("system", """You are an expert academic tutor. Create personalized educational content following these guidelines:
            
            Student Profile:
            - Class Level: {class_level}
            - Language: {language} 
            - Gender: {gender}
            
            Content Requirements:
            1. Use the audio/image/pdf if provided by the user
            2. Concise Summary notes
            3. Use bullet points and simple language appropriate for {class_level}
            4. Include practical examples and analogies
            5. Make it engaging and easy to understand
            6. If language is not English, provide content in {language}
            
            Additional Instructions: {user_instructions}"""),

            # ("user", "Please teach me about: {topic}"),
            
            ("human", [
                {"type": "text", "text": "Topic {topic}"},
                {"type": "file", "url": "{file_url}", "source_type": "url"}

            ])
        ])

        return template.invoke({
            "topic": self.topic,
            "class_level": self.student_profile.get("class_level", "general"),
            "language": self.student_profile.get("language", "English"),
            "gender": self.student_profile.get("gender", ""),
            # "word_limit": "150",
            "user_instructions": self.user_instructions or "Focus on key concepts with examples",
            "file_url": self.file_url
        })

In [16]:
def node_summary_notes(state: State):
    """LLM call to generate summary notes for student"""

    summary_prompt_template = PersonalizedPromptsStudent(
        state['user_prompt']['topic'], state['user_prompt']['file_url'], state['student_profile'])

    summary_prompt = summary_prompt_template.summary_notes()

    msg = model.invoke(summary_prompt)

    return {"summary_notes": msg.content}

In [17]:
input_state = {
    "student_profile": {
        "gender": "MALE",
        "class_level": "12th",
        "language": "HINDI"
    },
    "user_prompt": {
        "topic": "Uncertainty in AI",
        "file_url": "https://kclenjtsvdayeghmmmkm.supabase.co/storage/v1/object/public/learning-sources/public/Slides_Module8.pdf",
    }
}

In [18]:
output = node_summary_notes(input_state)

In [19]:
output

{'summary_notes': "ज़रूर, यहाँ आपके लिए AI में अनिश्चितता (Uncertainty) पर नोट्स दिए गए हैं:\n\n**AI में अनिश्चितता (Uncertainty in AI)**\n\nअनिश्चितता का मतलब है कि AI सिस्टम को हमेशा यह नहीं पता होता है कि क्या हो रहा है या आगे क्या होगा। इसे मापने और संभालने के तरीके यहाँ दिए गए हैं:\n\n**1. अनिश्चितता में काम करना (Acting under Uncertainty)**\n   - AI एजेंटों को अनिश्चित स्थितियों से निपटना होता है।\n   - एजेंट कभी भी निश्चित नहीं हो सकते कि वे किस स्थिति में हैं या कार्यों के बाद कहाँ पहुँचेंगे।\n   - एजेंट 'बिलीफ स्टेट' को ट्रैक करके अनिश्चितता को संभालते हैं।\n\n   *उदाहरण:*\n   - एक ऑटोमेटेड टैक्सी को हवाई अड्डे पर समय पर एक यात्री को पहुँचाना है, लेकिन उसे ट्रैफिक या अन्य अप्रत्याशित देरी का सामना करना पड़ सकता है।\n\n**2. बुनियादी संभाव्यता नोटेशन (Basic Probability Notation)**\n   - संभाव्यता (Probability) का उपयोग अनिश्चितता को स्पष्ट और सटीक रूप से व्यक्त करने के लिए किया जाता है।\n   - यह जटिल मॉडल बनाने और गणनाओं को सरल बनाने में मदद करता है।\n\n**3. फुल जॉइंट डिस्ट्रीब्